# **PCVK**

# Final Project

# **TI-3D**

## **Kelompok 3**


- ### **Aura Sagita Ashilah (2141720102)**
- ### **Bryan Toshin Saputro (2141720220)**
- ### **Masfudin Dwi Cahyo (2141720010)**


---



# **Import Library**

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import zipfile